In [1]:
import pandas as pd
import numpy as np
from os.path import exists
import re

In [2]:
## FUNCTIONS ##

#Parse String
def split_ele(x):
    a = np.array(re.findall(r"[-+]?\d*\.?\d+|[-+]?\d+", x))
    return a.astype(float)

In [3]:
# READ IN FILE ##
pose_landmark = ['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 
                 'RIGHT_EYE_OUTER', 'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 
                 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 
                 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX', 'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 
                 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX',
                 'RIGHT_FOOT_INDEX']

In [4]:
song_name_list = ['love_girl', 'party_rock_anthem', 'lite_spots', 'girlfriends_best_friend', 'scaredy_cat', 'girl_anachronism', 'lil_darlin', 'el_invento', 'test_drive', 'weight_in_gold', 'easy', 'comethru', 'classic', 'tongue_tied', 'motley', 'knife_talk', 'feed_the_fire', 'lose_it', 'thats_what_i_like', 'humble', 'skeletons', 'stairway_to_heaven', 'slow_dancing_in_the_dark', 'peaches', 'kilby_girl', 'jupiter_love', 'woman', 'get_up', 'throwin_elbows', 'power', 'ymca']
artist_name_list = ['cnblue', 'lmfao', 'kaytranada', 'walkabout', 'dpr_ian', 'the_dresden_dolls', 'count_basie', 'jose_gonzalez_calvaire', 'ariana_grande', 'gallant', 'wheein', 'jeremy_zucker', 'mkto', 'grouplove', 'post_malone', 'drake', 'lucky_daye', 'swmrs', 'bruno_mars', 'kendrick_lamar', 'keshi', 'led_zeppelin', 'joji', 'justin_bieber', 'backseat_lovers', 'trey_songz', 'doja_cat', 'shinedown', 'excision', 'kanye', 'village_people']
num_runs = 3
num_splits = 3


In [5]:
for file_num in range(1, num_runs+1):
    for index in range(len(song_name_list)):

        file_song = song_name_list[index]
        file_artist = artist_name_list[index]

        file_name = "data/"+str(file_num) + "_" + file_song + "_"+file_artist+".csv"


        #if file does not exist, skip it
        if(exists(file_name)):
            num_features = 10
            num_nodes = 14
            num_samples = 10

            pose_landmark_subset = ['LEFT_EYE_OUTER', 'RIGHT_EYE_OUTER', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 
                                    'LEFT_ELBOW', 'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_HIP', 'RIGHT_HIP',
                                    'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE']
            df_columns = ['LEFT_EYE_OUTER_POS', 'LEFT_EYE_OUTER_DIST', 'RIGHT_EYE_OUTER_POS', 'RIGHT_EYE_OUTER_DIST', 
                'LEFT_SHOULDER_POS', 'LEFT_SHOULDER_DIST', 'RIGHT_SHOULDER_POS', 'RIGHT_SHOULDER_DIST', 
                'LEFT_ELBOW_POS', 'LEFT_ELBOW_DIST', 'RIGHT_ELBOW_POS', 'RIGHT_ELBOW_DIST', 'LEFT_WRIST_POS', 
                'LEFT_WRIST_DIST', 'RIGHT_WRIST_POS', 'RIGHT_WRIST_DIST', 'LEFT_HIP_POS', 'LEFT_HIP_DIST', 
                'RIGHT_HIP_POS', 'RIGHT_HIP_DIST', 'LEFT_KNEE_POS', 'LEFT_KNEE_DIST', 'RIGHT_KNEE_POS', 
                'RIGHT_KNEE_DIST', 'LEFT_ANKLE_POS', 'LEFT_ANKLE_DIST', 'RIGHT_ANKLE_POS', 'RIGHT_ANKLE_DIST']
            df = pd.read_csv(file_name, sep = ',', usecols=[4, 7, 12, 13, 14, 15, 16, 17, 24, 25, 26, 27, 28, 29])

            splitDf = np.array_split(df, num_splits)
            
            for subDf in range(len(splitDf)):
                x_data = pd.DataFrame(columns=df_columns, index=range(len(splitDf[subDf])))
                y_data = pd.DataFrame(columns=df_columns, index=range(len(splitDf[subDf])))

                for node in pose_landmark_subset:
                    curr = splitDf[subDf][node]
                    vals = [split_ele(x) for x in curr]
                    for row in range(len(vals)):
                        colname_pos = node + "_POS"
                        colname_dist = node + "_DIST"

                        if(row == 0):
                            x_data[colname_dist][row] = 0
                            y_data[colname_dist][row] = 0
                        else:
                            x_data[colname_dist][row] = vals[row][0] - vals[row-1][0]
                            y_data[colname_dist][row] = vals[row][1] - vals[row-1][1]

                        x_data[colname_pos][row] = vals[row][0]
                        y_data[colname_pos][row] = vals[row][1]
                        
                x_data_file_write = f'preprocessed/{file_num}_{subDf}_{file_song}_{file_artist}_XDATA.csv'
                x_data.to_csv(x_data_file_write)
                y_data_file_write = f'preprocessed/{file_num}_{subDf}_{file_song}_{file_artist}_YDATA.csv'
                y_data.to_csv(y_data_file_write) 
    